# Python で気軽に化学・化学工学
# 第 5 章 データセットを前処理して扱いやすくする
## 5.3 類似した特徴量の組における一方の特徴量の削除

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット (descriptors_all_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、200 の分子記述子 (特徴量) で化学構造が数値化されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_all_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 1:] # 沸点以外の、分子構造の特徴量のみを x とします

In [ ]:
x # 念のため確認

In [ ]:
x.corr() # 相関行列の確認

相関係数の値の中に NaN があります。標準偏差が 0 の特徴量においては、相関係数を計算することができないため、本来であれば相関係数の値が入るところが NaN になっています。相関係数の計算の前に標準偏差が 0 の特徴量を削除するため、5.2 節で計算した結果である `deleting_variable_numbers_in_same_values.csv` (同じ値をもつサンプルの割合が大きい特徴量の結果) を使用します。まだ 5.2 節のサンプル Notebook を実行していない方は、実行してください。

In [ ]:
deleting_variable_numbers_in_same_values_df = pd.read_csv('deleting_variable_numbers_in_same_values.csv', index_col=0, header=0) # 削除する特徴量の読み込み (5.2 節を実行する必要があります)

In [ ]:
x = x.drop(deleting_variable_numbers_in_same_values_df.index, axis=1) # 特徴量の削除

In [ ]:
x # 念のため確認

In [ ]:
x.corr() # 念のため相関行列の確認

相関係数の絶対値が閾値以上の特徴量の組の一方の削除

In [ ]:
r_in_x = x.corr() # 相関行列

In [ ]:
r_in_x = abs(r_in_x) # 絶対値を取り、0 から 1 の間にする

相関行列における対角線の要素は自分自身との相関係数であり 1 です。他の特徴量と完全に (正もしくは負に) 相関している意味での相関係数の絶対値 1 と区別するため、対角線の要素を 0 とします。

In [ ]:
for i in range(r_in_x.shape[0]):
    r_in_x.iloc[i, i] = 0 # 相関行列における対角線の要素を 0 にします

In [ ]:
threshold_of_r = 0.95 # 相関係数の絶対値の閾値

In [ ]:
deleting_variable_numbers_in_r = []  # 空の list の変数を準備しておき、ここに相関係数の絶対値が閾値以上となる特徴量の一方の番号を入れていきます

In [ ]:
for i in range(r_in_x.shape[0]):
    r_max = r_in_x.max() # 特徴量ごとの、他の特徴量との間における相関係数の絶対値の最大値
    r_max = list(r_max) # list に変換
    if max(r_max) >= threshold_of_r: # 相関係数の絶対値が閾値以上の特徴量の組があるとき
        variable_number_1 = r_max.index(max(r_max)) # 相関係数の絶対値が最大となる特徴量の組における、一方の番号
        r_in_variable_1 = list(r_in_x.iloc[:, variable_number_1]) # 上で選ばれた特徴量における相関係数の最大値
        variable_number_2 = r_in_variable_1.index(max(r_in_variable_1)) # 相関係数の絶対値が最大となる特徴量の組における、もう一方の番号
        # variable_number_1 を削除するか、variable_number_2 を削除するか、他の特徴量との相関係数の絶対値の和を計算し、それが大きい方とします
        r_sum_1 = r_in_x.iloc[:, variable_number_1].sum()
        r_sum_2 = r_in_x.iloc[:, variable_number_2].sum()
        if r_sum_1 >= r_sum_2:
            delete_x_number = variable_number_1
        else:
            delete_x_number = variable_number_2
        deleting_variable_numbers_in_r.append(delete_x_number) # 削除する特徴量の番号を追加
        # 削除する特徴量の影響をなくすため、対応する相関係数の絶対値を 0 に
        r_in_x.iloc[:, delete_x_number] = 0
        r_in_x.iloc[delete_x_number, :] = 0
    else: # 相関係数の絶対値が閾値以上の特徴量の組がなくなったら終了
        break

In [ ]:
deleting_variable_numbers_in_r # 念のため確認

In [ ]:
len(deleting_variable_numbers_in_r) # 削除される特徴量の数

In [ ]:
deleting_variable_numbers_in_r_df = pd.DataFrame(deleting_variable_numbers_in_r) # DataFrame 型に変換

In [ ]:
x.columns[deleting_variable_numbers_in_r]

In [ ]:
deleting_variable_numbers_in_r_df.index = x.columns[deleting_variable_numbers_in_r]  # 列の名前を特徴量の名前に

In [ ]:
deleting_variable_numbers_in_r_df.columns = ['deleting variable numbers']

In [ ]:
deleting_variable_numbers_in_r_df # 念のため確認

In [ ]:
deleting_variable_numbers_in_r_df.to_csv('deleting_variable_numbers_in_r.csv')

In [ ]:
deleting_variable_numbers_in_r_df.index

In [ ]:
x.drop(deleting_variable_numbers_in_r_df.index, axis=1) # 特徴量の削除

In [ ]:
x_new = x.drop(deleting_variable_numbers_in_r_df.index, axis=1) # 特徴量を削除したものを x_new にする

In [ ]:
x_new # 念のため確認

In [ ]:
x_new.corr() # 相関行列の確認

相関係数の絶対値が `threshold_of_r` 以上の特徴量の組はないことが分かります

### 練習問題

データセット `descriptors_all_with_logs.csv` を読み込み、90 % 以上のサンプルで同じ値をもつ特徴量を削除し、その後に相関係数が 0.99 以上の特徴量ペアの一つを削除してから、特徴量の標準化をしましょう。一番下にコードの例があります。

### 水溶解度のデータセット (descriptors_all_with_logs.csv)
T. J.	Hou et al. が収集した[水溶解度のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci034184n)。1290 個の化合物について、水溶解度が測定されており、200 の分子記述子 (特徴量) で化学構造が数値化されています。目的変数である logS とは、水への溶解度を S [mol/L] としたときの log(S) のことです。

### 練習問題 コードの例

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_all_with_logs.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 1:] # logS 以外の、分子構造の特徴量のみを x とします

In [ ]:
x # 念のため確認

同じ値をもつサンプルの割合が大きい特徴量の削除

In [ ]:
threshold_of_rate_of_same_values = 0.9 # 同じ値をもつサンプルの割合の閾値

In [ ]:
deleting_variable_numbers_in_same_values = [] # 空の list の変数を準備しておき、ここに同じ値をもつサンプルの割合が閾値以上となる特徴量の番号を入れていきます

In [ ]:
for x_number in range(x.shape[1]):
    same_value_numbers = x.iloc[:, x_number].value_counts()
    if same_value_numbers.iloc[0] / x.shape[0] >= threshold_of_rate_of_same_values:
        deleting_variable_numbers_in_same_values.append(x_number)

In [ ]:
deleting_variable_numbers_in_same_values # 念のため確認

In [ ]:
len(deleting_variable_numbers_in_same_values) # 削除される特徴量の数

In [ ]:
deleting_variable_numbers_in_same_values_df = pd.DataFrame(deleting_variable_numbers_in_same_values) # DataFrame 型に変換

In [ ]:
deleting_variable_numbers_in_same_values_df.index = x.columns[deleting_variable_numbers_in_same_values]  # 列の名前を特徴量の名前に

In [ ]:
deleting_variable_numbers_in_same_values_df.columns = ['deleting variable numbers']

In [ ]:
deleting_variable_numbers_in_same_values_df # 念のため確認

In [ ]:
deleting_variable_numbers_in_same_values_df.to_csv('deleting_variable_numbers_in_same_values_logs.csv')

In [ ]:
deleting_variable_numbers_in_same_values_df.index

In [ ]:
x = x.drop(deleting_variable_numbers_in_same_values_df.index, axis=1) # 特徴量を削除したものを x にする

In [ ]:
x # 念のため確認

相関係数の絶対値が閾値以上の特徴量の組の一方の削除

In [ ]:
r_in_x = x.corr() # 相関行列

In [ ]:
r_in_x = abs(r_in_x) # 絶対値を取り、0 から 1 の間にする

In [ ]:
for i in range(r_in_x.shape[0]):
    r_in_x.iloc[i, i] = 0 # 相関行列における対角線の要素を 0 にします

In [ ]:
threshold_of_r = 0.99 # 相関係数の絶対値の閾値

In [ ]:
deleting_variable_numbers_in_r = []  # 空の list の変数を準備しておき、ここに相関係数の絶対値が閾値以上となる特徴量の一方の番号を入れていきます

In [ ]:
for i in range(r_in_x.shape[0]):
    r_max = r_in_x.max() # 特徴量ごとの、他の特徴量との間における相関係数の絶対値の最大値
    r_max = list(r_max) # list に変換
    if max(r_max) >= threshold_of_r: # 相関係数の絶対値が閾値以上の特徴量の組があるとき
        variable_number_1 = r_max.index(max(r_max)) # 相関係数の絶対値が最大となる特徴量の組における、一方の番号
        r_in_variable_1 = list(r_in_x.iloc[:, variable_number_1]) # 上で選ばれた特徴量における相関係数の最大値
        variable_number_2 = r_in_variable_1.index(max(r_in_variable_1)) # 相関係数の絶対値が最大となる特徴量の組における、もう一方の番号
        # variable_number_1 を削除するか、variable_number_2 を削除するか、他の特徴量との相関係数の絶対値の和を計算し、それが大きい方とします
        r_sum_1 = r_in_x.iloc[:, variable_number_1].sum()
        r_sum_2 = r_in_x.iloc[:, variable_number_2].sum()
        if r_sum_1 >= r_sum_2:
            delete_x_number = variable_number_1
        else:
            delete_x_number = variable_number_2
        deleting_variable_numbers_in_r.append(delete_x_number) # 削除する特徴量の番号を追加
        # 削除する特徴量の影響をなくすため、対応する相関係数の絶対値を 0 に
        r_in_x.iloc[:, delete_x_number] = 0
        r_in_x.iloc[delete_x_number, :] = 0
    else: # 相関係数の絶対値が閾値以上の特徴量の組がなくなったら終了
        break

In [ ]:
deleting_variable_numbers_in_r # 念のため確認

In [ ]:
len(deleting_variable_numbers_in_r) # 削除される特徴量の数

In [ ]:
deleting_variable_numbers_in_r_df = pd.DataFrame(deleting_variable_numbers_in_r) # DataFrame 型に変換

In [ ]:
deleting_variable_numbers_in_r_df.index = x.columns[deleting_variable_numbers_in_r]  # 列の名前を特徴量の名前に

In [ ]:
deleting_variable_numbers_in_r_df.columns = ['deleting variable numbers']

In [ ]:
deleting_variable_numbers_in_r_df # 念のため確認

In [ ]:
deleting_variable_numbers_in_r_df.to_csv('deleting_variable_numbers_in_r_logs.csv')

In [ ]:
x = x.drop(deleting_variable_numbers_in_r_df.index, axis=1) # 特徴量を削除したものを x にする

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # 平均を引いてから、標準偏差で割ります。x は DataFrame 型、x.mean(), x.std() は Series 型でデータ型は異なりますが、特徴量の名前が同じであるため、x のすべてのサンプルに対して x.mean() を引き、x.std() で割る計算になります。

In [ ]:
autoscaled_x # 念のため確認